In [1]:
import ee
import datetime
import pandas as pd
import time

# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

def get_elevation_and_slope(lat, lon):
    point = ee.Geometry.Point([lon, lat])

    elevation_image = ee.Image('USGS/SRTMGL1_003')
    slope_image = ee.Terrain.slope(elevation_image)

    elevation = elevation_image.reduceRegion(ee.Reducer.first(), point, 30).get('elevation') # 30 m native resolution of the dataset, to get the most precise value
    slope = slope_image.reduceRegion(ee.Reducer.first(), point, 30).get('slope')

    elevation_info = elevation.getInfo()
    slope_info = slope.getInfo()

    if elevation_info is None or slope_info is None:
        return None, None

    return elevation_info, slope_info

df = pd.read_csv("mapdata_wildfires.csv")
df_error = pd.read_csv("weather_data_wildfires_error.csv")


coordinates = []
coordinates_error = []

for _, row in df.iterrows():
    lat = row['incident_latitude']
    lon = row['incident_longitude']
    coordinates.append((lat, lon))
    
for _, row in df_error.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    coordinates_error.append((lat, lon))

elevation_slope_values = []

for lat, lon in coordinates_error: # Change for full dataset, remove _error
    elevation, slope = get_elevation_and_slope(lat, lon)
    elevation_slope_values.append((elevation, slope))

feature_list = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(lon, lat), {'Latitude': lat, 'Longitude': lon, 'Elevation': elevation, 'Slope': slope})
    for (lat, lon), (elevation, slope) in zip(coordinates_error, elevation_slope_values) # Change for full dataset, remove _error
])

task_config = {
    'folder': 'EarthEngineExports',
    'fileFormat': 'CSV',
    'driveFileNamePrefix': 'elevation_slope_data_wildfires',
    'selectors': ['Latitude', 'Longitude', 'Elevation', 'Slope']
}

task = ee.batch.Export.table.toDrive(feature_list, **task_config)
task.start()

while task.status()['state'] in ['READY', 'RUNNING']:
    print('Exporting elevation and slope data...')
    time.sleep(30)

print('Elevation and slope data export completed.')


Enter verification code: 4/1AVHEtk4XZIFr6qIL-2jRazkT0e7ux41pp_K1rmF7tYpf6P8iOogztQTzH8U

Successfully saved authorization token.
Exporting elevation and slope data...
Exporting elevation and slope data...
Exporting elevation and slope data...
Elevation and slope data export completed.
